

   <b> "Metaclasses are deeper magic than 99% of users should ever worry about. If you wonder whether you need them, you don’t (the people who actually need them know with certainty that they need them, and don’t need an explanation about why)".</b>

    – Tim Peters


<img src=../../resources/metaclass.png>

 ### <em>Everything</em> in python is an object: it turns out that this is true of <em>classes themselves</em>.

In [ ]:
class DoNothing(object):
    pass

In [ ]:
d = DoNothing()
type(d)

In [ ]:
L = [1, 2, 3]
type(L)

In [ ]:
type(DoNothing)

In [ ]:
type(tuple), type(list), type(int), type(float)

### <em>classes are objects</em>, and they are objects of type <em>```type```</em>
#### <code>type</code> is a <em>metaclass</em>: a class which instantiates classes.

.

.

.

.

.

.

.

.

.

### Metaprogramming: Creating Classes on the Fly

What does a typical function do?

 Take some arguments, do some operations, and create & return an object. 
 
 
Then it should be possible to create an object of type <em>type</em> (that is, a class), and returning that


 

.

.

.

.

.

.

In [ ]:
def class_factory():
    class Foo(object):
        pass
    return Foo

F = class_factory()
f = F()
print(type(f))

We can accomplish this by instantiating Foo from type directly.

help(type)

Help on class type in module __builtin__:

class type(object)
 |  type(object) -> the object's type
 |  type(name, bases, dict) -> a new type

In [ ]:
class MyClass(object):
    pass

In [ ]:
MyClass = type('MyClass', (), {})

type(name, bases, dct)

<ul>
<li><code>name</code> is a string giving the name of the class to be constructed</li>
<li><code>bases</code> is a tuple giving the parent classes of the class to be constructed</li>
<li><code>dct</code> is a dictionary of the attributes and methods of the class to be constructed</li>
</ul>

In [ ]:


class Foo(object):
    i = 4

class Bar(Foo):
    def get_i(self):
        return self.i
    
b = Bar()
print(b.get_i())



In [ ]:


Foo = type('Foo', (), dict(i=4))

Bar = type('Bar', (Foo,), dict(get_i = lambda self: self.i))

b = Bar()
print(b.get_i())



.

.

.

.

.

.

.

.

.



#### Just as we can inherit from and extend a class we've created, we can also:
        
        - inherit from and extend the type metaclass
        - create custom behavior in our metaclass 
        - fry our brains..

.

.

.

.

.

.

In [ ]:
class InterfaceMeta(type):
    def __new__(cls, name, parents, dct):
        # create a class_id if it's not specified
        if 'class_id' not in dct:
            dct['class_id'] = name.lower()
        
        # open the specified file for writing
        if 'file' in dct:
            filename = dct['file']
            dct['file'] = open(filename, 'w')
        
        # we need to call type.__new__ to complete the initialization
        return super(InterfaceMeta, cls).__new__(cls, name, parents, dct)

In [ ]:
Interface = InterfaceMeta('Interface', (), dict(file='tmp.txt'))

print(Interface.class_id)
print(Interface.file)

In [ ]:
class Interface(object):
    __metaclass__ = InterfaceMeta
    file = 'tmp1.txt'
    
print(Interface.class_id)
print(Interface.file)



In [ ]:
type(Interface)

#### Another example: Registering Subclasses

In [ ]:


class DBInterfaceMeta(type):
    # we use __init__ rather than __new__ here because we want
    # to modify attributes of the class *after* they have been
    # created
    def __init__(cls, name, bases, dct):
        if not hasattr(cls, 'registry'):
            # this is the base class.  Create an empty registry
            cls.registry = {}
        else:
            # this is a derived class.  Add cls to the registry
            interface_id = name.lower()
            cls.registry[interface_id] = cls
            
        super(DBInterfaceMeta, cls).__init__(name, bases, dct)



In [ ]:
class DBInterface(object):
    __metaclass__ = DBInterfaceMeta
    
print(DBInterface.registry)

In [ ]:
class FirstInterface(DBInterface):
    pass

class SecondInterface(DBInterface):
    pass

class SecondInterfaceModified(SecondInterface):
    pass

print(DBInterface.registry)

e.g.: https://github.com/django/django/blob/master/django/db/models/base.py

### Abstract Classes

In [ ]:
from abc import ABCMeta, abstractmethod

class Vehicle(object):
    """A vehicle for sale by Jeffco Car Dealership.


    Attributes:
        wheels: An integer representing the number of wheels the vehicle has.
        miles: The integral number of miles driven on the vehicle.
        make: The make of the vehicle as a string.
        model: The model of the vehicle as a string.
        year: The integral year the vehicle was built.
        sold_on: The date the vehicle was sold.
    """

    __metaclass__ = ABCMeta

    base_sale_price = 0

    def sale_price(self):
        """Return the sale price for this vehicle as a float amount."""
        if self.sold_on is not None:
            return 0.0  # Already sold
        return 5000.0 * self.wheels

    def purchase_price(self):
        """Return the price for which we would pay to purchase the vehicle."""
        if self.sold_on is None:
            return 0.0  # Not yet sold
        return self.base_sale_price - (.10 * self.miles)

    @abstractmethod
    def vehicle_type():
        """"Return a string representing the type of vehicle this is."""
        pass

In [ ]:
class Car(Vehicle):
    """A car for sale by Jeffco Car Dealership."""

    base_sale_price = 8000
    wheels = 4

    def vehicle_type(self):
        """"Return a string representing the type of vehicle this is."""
        return 'car'

class Truck(Vehicle):
    """A truck for sale by Jeffco Car Dealership."""

    base_sale_price = 10000
    wheels = 4

    def vehicle_type(self):
        """"Return a string representing the type of vehicle this is."""
        return 'truck'